In [1]:
with open('input.txt','r',encoding='utf-8') as f:
    text = f.read()

In [2]:
print("Length of Dataset: ", len(text))

Length of Dataset:  98595


In [3]:
print(text[:100])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [4]:
# We are extracting all the unique characters in the text, in toa, we call it the alphabets SIGMA
# We are trying to build a character level language model
chars = sorted(list(set(text)))
voca_size =len(chars)
print("".join(chars))
print(voca_size)


 !&',-.:;?ABCDEFGHIJKLMNOPQRSTUVWYabcdefghijklmnopqrstuvwxyz
61


In [5]:
stoi = {ch:i for i, ch in enumerate(chars)} 
itos = {i:ch for i, ch in enumerate(chars)}

encode = lambda s: [stoi[c] for c in s]

decode = lambda l: ''.join([itos[i] for i in l])


In [6]:
import torch
data = torch.tensor(encode(text),dtype=torch.long)
print(data.shape,data.dtype)
print(data[:1000])

torch.Size([98595]) torch.int64
tensor([16, 43, 52, 53, 54,  1, 13, 43, 54, 43, 60, 39, 48,  8,  0, 12, 39, 40,
        49, 52, 39,  1, 57, 39,  1, 50, 52, 49, 37, 39, 39, 38,  1, 35, 48, 59,
         1, 40, 55, 52, 54, 42, 39, 52,  5,  1, 42, 39, 35, 52,  1, 47, 39,  1,
        53, 50, 39, 35, 45,  7,  0,  0, 11, 46, 46,  8,  0, 29, 50, 39, 35, 45,
         5,  1, 53, 50, 39, 35, 45,  7,  0,  0, 16, 43, 52, 53, 54,  1, 13, 43,
        54, 43, 60, 39, 48,  8,  0, 34, 49, 55,  1, 35, 52, 39,  1, 35, 46, 46,
         1, 52, 39, 53, 49, 46, 56, 39, 38,  1, 52, 35, 54, 42, 39, 52,  1, 54,
        49,  1, 38, 43, 39,  1, 54, 42, 35, 48,  1, 54, 49,  1, 40, 35, 47, 43,
        53, 42, 10,  0,  0, 11, 46, 46,  8,  0, 28, 39, 53, 49, 46, 56, 39, 38,
         7,  1, 52, 39, 53, 49, 46, 56, 39, 38,  7,  0,  0, 16, 43, 52, 53, 54,
         1, 13, 43, 54, 43, 60, 39, 48,  8,  0, 16, 43, 52, 53, 54,  5,  1, 59,
        49, 55,  1, 45, 48, 49, 57,  1, 13, 35, 43, 55, 53,  1, 23, 35, 52, 37,
        

In [7]:
# Let's split our data into train and validation split
n  =int(0.9 * len(data)) # first 90% will be train dataset
train_data = data[:n]
val_data = data[n:]

# Splitting the data like this, helps us to what degree/level our model is overfitting or underfitting

In [8]:
len(val_data), len(train_data)

(9860, 88735)

In [9]:
block_size = 8  # This is the max context length, we just happen to take 8
train_data[:block_size+1]

tensor([16, 43, 52, 53, 54,  1, 13, 43, 54])

In [10]:
x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target=y[t]
    print(f"when input is {context} the target: {target}")

when input is tensor([16]) the target: 43
when input is tensor([16, 43]) the target: 52
when input is tensor([16, 43, 52]) the target: 53
when input is tensor([16, 43, 52, 53]) the target: 54
when input is tensor([16, 43, 52, 53, 54]) the target: 1
when input is tensor([16, 43, 52, 53, 54,  1]) the target: 13
when input is tensor([16, 43, 52, 53, 54,  1, 13]) the target: 43
when input is tensor([16, 43, 52, 53, 54,  1, 13, 43]) the target: 54


In [11]:
torch.manual_seed(1337)
batch_size = 4 # How many independent sequences will we process in parallel
block_size =8 #What is the maximum context length to make those predictions



In [25]:
def get_batch(split):
    #generate a small batch of data of inputs x and targets y
    data = train_data  if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size, (batch_size,))
    x= torch.stack([data[i:i+block_size] for i in ix])
    y =torch.stack([data[i+1:i+block_size] for i in ix])
    return x,y

xb,yb =get_batch('train')
print('inputs:')
print(xb.shape)
print(xb)
print('targets:')
print(yb.shape)
print(yb)
print('-------')

inputs:
torch.Size([4, 8])
tensor([[49, 54,  1, 54, 42, 39,  1, 37],
        [38,  1, 36, 39,  1, 43, 48,  1],
        [40, 43, 48, 38,  1, 54, 42, 39],
        [38,  1, 54, 42, 55, 53,  1, 35]])
targets:
torch.Size([4, 7])
tensor([[54,  1, 54, 42, 39,  1, 37],
        [ 1, 36, 39,  1, 43, 48,  1],
        [43, 48, 38,  1, 54, 42, 39],
        [ 1, 54, 42, 55, 53,  1, 35]])
-------
